# P05 schedule

## Preamble

This section contains package imports and general settings for the notebook.

In [17]:
import pandas as pd
import datetime as dt
import calendar
import numpy as np
from pathlib import Path
import os

pd.set_option('display.max_columns', 50) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', 400) #replace n with the number of rows you want to see completely

## User input
- Year 
- Path to the dirctory which contains the following documents:
    - Petra 3 schedule provided by Oliver Seeck
    - DOOR scheduling support - available after the ranking was confirmed in DOOR

In [18]:
year = 2023

dc_path = Path('desycloud/documents/' + str(year) + '/p05_schedule/')  # directory which contains all necessary files to import, startng from desycloud
dc_p3sch_path =  dc_path / 'Schedule2023_draft_Status2022.08.18_shifted_timing.xlsx'  # path to Petra 3 schedule
dc_proposals_path = dc_path / '2023_1/door.scheduling.support.xlsx'  # path to DOOR scheduling support

# School holidays in Hamburg, Niedersachsen and Schleswig-Holstein
school_holidays = {"Hamburg" :
            { "Weihnachtferien" :  pd.date_range(start="2023-01-01", end="2023-01-06"), 
              "Winterferien" : pd.date_range(start="2023-01-27", end="2023-01-27"),
              "Osterferien" : pd.date_range(start="2023-03-06", end="2023-03-17"),
              "Pfingstferien" : pd.date_range(start="2023-05-15", end="2023-05-19"),
              "Sommerferien" : pd.date_range(start="2023-07-13", end="2023-08-23"),
              "Herbstferien" : pd.date_range(start="2023-10-02", end="2023-10-27")},
            "Niedersachsen" :
            { "Weihnachtferien" :  pd.date_range(start="2023-01-01", end="2023-01-06"),
              "Winterferien" : pd.date_range(start="2023-01-30", end="2023-01-31"),
              "Osterferien" : pd.date_range(start="2023-03-27", end="2023-04-11"),
              "Pfingstferien" : pd.date_range(start="2023-05-19", end="2023-05-30"),
              "Sommerferien" : pd.date_range(start="2023-07-06", end="2023-08-16"),
              "Herbstferien1" : pd.date_range(start="2023-10-02", end="2023-10-02"),
              "Herbstferien2" : pd.date_range(start="2023-10-16", end="2023-10-30")},
            "Schleswig-Holstein" :
            { "Weihnachtferien" :  pd.date_range(start="2023-01-01", end="2023-01-07"),
              "Osterferien" : pd.date_range(start="2023-04-06", end="2023-04-22"),
              "Pfingstferien" : pd.date_range(start="2023-05-19", end="2023-05-20"),
              "Sommerferien" : pd.date_range(start="2023-07-17", end="2023-08-26"),
              "Herbstferien" : pd.date_range(start="2023-10-16", end="2023-10-27")}
           }

# Holidays in Hamburg
hh_holidays =  {"Neujahr" :  dt.date(2023,1,1),
                "Karfreitag" : dt.date(2023,4,7),
                "Ostermontag" : dt.date(2023,4,10),
                "Tag der Arbeit" : dt.date(2023,5,1),
                "Christi Himmelfahrt" : dt.date(2023,5,18),
                "Pingstmontag" : dt.date(2023,5,29),
                "Tag der Deutschen Einheit" : dt.date(2023,10,3),
                "Reformationstag" : dt.date(2023,10,31),
                "1. Weihnachtsfeiertag" : dt.date(2023,12,25),
                "2. Weihnachtsfeiertag" : dt.date(2023,12,26)
               }

In [19]:
# Choose the path depending if I'm at home or at work
system = os.name
if os.name == "nt":
    path = "D:/" / dc_path
    p3sch_path =  "D:/" / dc_p3sch_path
    proposals_path = "D:/" / dc_proposals_path
else:
    path = "/home/fwilde/" / dc_path
    p3sch_path =  "/home/fwilde/" / dc_p3sch_path
    proposals_path = "/home/fwilde/" / dc_proposals_path
    
path

PosixPath('/home/fwilde/desycloud/documents/2023/p05_schedule')

## Import Petra 3 schedule

#### Scripts to filter imported tables and generate empty p05 schedule

In [44]:
##################################################
# Class to work with the p05 schedule
##################################################

class p05sch:
    """
    P05 schedule class.

    Args:
        year <int>: Year for which the p05 schedule should be created
        p3sch_path <str>: path to the PETRA III schedule
        school_holidays <optional, dict>: dictionary containing the school holidays {Bundesland: {Ferienname: daterange}, ...}
        hh_holdays <optional, dict>: dictionary containing the Hamburg holdiays {Feiertagname: date, ...}
    """
    def __init__(self, year, p3sch_path, school_holidays=None, hh_holidays=None, proposals_path=None):
        self.year = year
        self.p05_schedule = None
        self.p3_modes = None
        self.p3_addinfo = None
        self.p3sch = pd.read_excel(p3sch_path)
        self.filter_p3sch()
        self.school_holidays = school_holidays
        self.hh_holidays = hh_holidays

        if not self.p05_schedule:
            self.init_p05_schedule()

    def filter_p3sch(self):
        '''
        Strip imported Petra 3 schedule of everything except the Petra III modes and additional info.

        Args:
            p3sch <pandas.DataFrame>:  Petra III schedule read in by read_excel() 

        Return:
            p3_modes <pandas.DataFrame>:  table with PETRA III modes 
            p3_addinfo <pandas.DataFrame>:  table with additional info 
        '''
        counter=0
        machine_col = []
        addinfo_col = []
        months = ["December", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December.1"]
        for col in self.p3sch.iloc[[0]]:
            if col in months:
                machine_col.append(counter + 3)  # 3rd column after the month name column should contain the Petra III modes
                addinfo_col.append(counter + 4)  # 4th column after the month name column should contain the Petra III additional info
            counter +=1
        self.p3_modes = self.p3sch[self.p3sch.columns[machine_col]][1:32]  # fetch mode info from Petra III schedule. Everything beyond row 32 should not belong to the schedule.
        self.p3_addinfo = self.p3sch[self.p3sch.columns[addinfo_col]][1:32]  # fetch additional info from Petra III schedule. Everything beyond row 32 should not belong to the schedule.

        self.p3_modes.columns = months # set column names to months
        self.p3_addinfo.columns = months # set column names to months

        self.p3_modes.drop(["December"], axis=1, inplace=True)  # get rid of last years December column
        self.p3_addinfo.drop(["December"], axis=1, inplace=True)  # get rid of last years December column
        self.p3_modes.rename(columns={"December.1":"December"}, inplace=True)  # rename imported December.1 of current year to December
        self.p3_addinfo.rename(columns={"December.1":"December"}, inplace=True)  # rename imported December.1 of current year to December

    def schedule_inject_data(self, startdate, enddate, column, data):
        '''
        Injects data into a column of the schedule. Use data=np.nan to remove data from a data frame.

        Args:
            schedule <pandas.DataFrame>
            startdate <datetime.date>
            enddate <datetime.date>
            column <string>
            data <arbitrary>

        Return:
            True
        '''
        dr = pd.date_range(startdate, enddate)
        for date in dr:
            self.p05_schedule.loc[date, column] = data

    def schedule_inject_list(self, startdate, enddate, column, data):
        '''
        Injects list data into a column of the schedule. If the list is shorter than the time range,
        missing values will be set to NaN.

        Args:
            startdate <datetime.date>
            enddate <datetime.date>
            column <string>
            data <list> <np.array> <pd.Series>

        Return:
            True
        '''
        dr = pd.date_range(startdate, enddate)  # generate a pandas series wiht dates
        lendiff = len(dr) - len(data) 
        if lendiff > 0:  # if the data list is shorter than the list of dates, fill with the missing values with None
            try:
                data.append(None*lendiff)
            except:
                data = np.append(data, [None]*lendiff)

        if type(data) == list:  # in case data of type <list>, <np.array>
            for i, date in enumerate(dr):
                self.p05_schedule.loc[date, column] = data[i]
        if type(data) == pd.Series:
            for i, date in enumerate(dr):  # in case data of type <pd.Series>
                self.p05_schedule.loc[date, column] = data.iloc[i]

        self.p05_schedule.replace(to_replace=[None], value=np.nan, inplace=True)

    def init_p05_schedule(self):
        '''
        Generates an empty p05 schedule which only includes the Petra 3 modes / additional info.

        Args:
            year <int>

        Return:
            p05_schedule <pandas.DataFrame>
        '''
        startdate = dt.date(self.year,1,1)
        enddate = dt.date(self.year,12,31)
        months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        p05_schedule_columns = ['holidays', 'p3 modes', 'p3 info', 'type', 'Prop ID', 'app ID', 'leader', 'pi', 'local contact', 'absence', 'extra hours', 'short title', 'conferences', 'comment']
        self.p05_schedule = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=p05_schedule_columns)
        for month in np.arange(12):
            daysinmonth = calendar.monthrange(year, month+1)[1]  # get number of days in current month
            month_modes = self.p3_modes[months[month]].iloc[:daysinmonth]
            month_addinfo = self.p3_addinfo[months[month]].iloc[:daysinmonth]
            self.schedule_inject_list(dt.date(self.year, month+1, 1), dt.date(self.year, month+1, daysinmonth), "p3 modes", month_modes)
            self.schedule_inject_list(dt.date(self.year, month+1, 1), dt.date(self.year, month+1, daysinmonth), "p3 info", month_addinfo)

    def init_school_holiday_column(self, year):
        startdate = dt.date(year,1,1)
        enddate = dt.date(year,12,31)
        sh_col = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=["school holidays"])
        for day, holiday in sh_col.iterrows():  # 1. iterate over all days in the year, to compare each day with the school holidays
            holidays_in = ""  # this is the string withthe bundesland holidays on a specific days (like "HNS")
            for bundesland, b_school_holidays in school_holidays.items():  # 2. iterate over different bundesländer
                for holidayname, daterange in b_school_holidays.items():  # within each bundesland iterate over the
                    if day in daterange:
                        holidays_in += bundesland[0]  # 
            print(day, holidays_in)


            

##################################################
# Functions to work with table of proposals
##################################################
class proposals:
    """
    The proposals class can read, filter and modify a table with all P05 proposals.
    
    Args:
        proposals_path <str>: Path to the scheduling support table (list of proposals).
    """
    
    def __init__(self, proposals_path=None):
        self.proposal_table = pd.read_excel(proposals_path)
        self.filtered_proposal_table = None

    def filter_proposals(self, filter_string='auto', include=[]):
        '''
        Strip scheduling support table (list of proposal) from all but the current proposals.

        Args:
            filter_string <str>: String used to filter scheduling support table in the "Ranking" column. 
                                 If set to "auto", the table is filtered based on the
                                 highest sum of the call and year integers in the
                                 "Ranking" column (eg. "2nd call 2022" -> 2024).
            include <list>: Add a list of Application IDs, that should also be included in the filtered_proposal_table
        '''
        if filter_string == "auto":
            mask_petra = self.proposal_table["Ranking"].str.contains("PETRA")  # create mask based on the string "PETRA"
            mask_petra.fillna(False, inplace=True)  # Replace NaN with False in mask
            filtered_proposals = self.proposal_table[mask_petra]  # filter proposals based on mask_petra
            r_filtered_proposals = filtered_proposals.reset_index(drop = True)  # rebuild index from 0.. in filtered_proposals
            numbers_in_ranking = filtered_proposals["Ranking"].str.findall(r'([0-9]+)')  # new dataframe generating a list with all numbers (as strings) in the "Ranking" column
            split_numbers_in_ranking = pd.DataFrame(numbers_in_ranking.to_list(), columns = ['ranking', 'call', 'year'])  # new dataframe based o
            split_numbers_in_ranking["sum_call_year"] = split_numbers_in_ranking["call"].astype("int") + split_numbers_in_ranking["year"].astype("int")
            max_nir = max(split_numbers_in_ranking["sum_call_year"])
            split_numbers_in_ranking["mask"] = (split_numbers_in_ranking["sum_call_year"] == max_nir)
            mask_newest = split_numbers_in_ranking["mask"]
            self.filtered_proposal_table = r_filtered_proposals[mask_newest]
        else:
            mask_petra = proposals["Ranking"].str.contains(filter_string)  # create mask based on the filter_string
            mask_petra.fillna(False, inplace=True)  # Replace NaN with False in mask
            filtered_proposals = proposals[mask_petra]  # filter proposals based on mask_petra
            self.filtered_proposal_table = filtered_proposals.reset_index(drop = True)  # rebuild index from 0.. in filtered_proposals
        include_proposals = self.proposal_table[self.proposal_table["Application"].isin(include)].copy()
        include_proposals.loc[:, "Ranking (numbers)"] = -1
        # Add column sthat will be needed later on
        self.filtered_proposal_table = pd.concat([self.filtered_proposal_table, include_proposals])
        self.filtered_proposal_table.set_index("Application", inplace=True)  # prevent multiple identical Application IDs
        self.filtered_proposal_table.insert(8, "actual shifts", None)
        self.filtered_proposal_table.insert(9, "start date", None)
        self.filtered_proposal_table.insert(10, "local contact", None)
    
    def show_filtered_proposal_table(self, exclude=['Collaboration','Ranking', 'Beamline','Submitted','Filling mode (bunches)','Beam size']):
        """
        Show the filtered proposal table and optional exclude columns.
        
        Args:
            exclude <str list>: List of columns that should be excluded in display. Defaults to: 
            ['Collaboration','Ranking', 'Beamline','Submitted','Filling mode (bunches)','Beam size']
        """
        display(self.filtered_proposal_table.loc[:, ~self.filtered_proposal_table.columns.isin(exclude)].sort_values(by="Ranking (numbers)"))

    def assign_lc_shifts(self, appid, local_contact, actual_shifts, comment=None):
        """
        Assign an AppID a local contact and how many shifts they acutally get.
        
        Args:
            appid <int>: Application ID
            local_contact <str>: Local contact (initials)
            comment <optional, str>: A comment tha should go along with the beamtime
        """
        pass

### Initialize p05_schedule and proposal_table

In [45]:
p5sch_class = p05sch(2023, p3sch_path, school_holidays=school_holidays, hh_holidays=hh_holidays)

In [46]:
### This part only works, if the scheduling support (list of proposals) is available

#proposals = filter_proposals(proposals, filter_string="2nd call 2022")
proposals_class = proposals(proposals_path)
proposals_class.filter_proposals(include=[11016379, 11016663, 11016664,11016665,11016666,11016667,11016668])

In [47]:
proposals_class.show_filtered_proposal_table()

,Proposal,Setup,Ranking (numbers),Shifts applied,Shifts assigned,actual shifts,start date,local contact,Title,Leader,PI,Energy,Preferred dates,Unacceptable
Application,,,,,,,,,,,,,,
11016379,I-20221317,Microtomography (EH2),-1,6.0,6,None,None,None,Chromite grains in Antarctic micrometeorites: ...,Rout,Rout,18 keV,NaN,NaN
11016663,BAG-20210019,Microtomography (EH2),-1,12.0,72,None,None,None,Understanding different scales oft the ecosyst...,Wipfler,Hammel,18 keV,NaN,NaN
11016668,BAG-20211055,Microtomography (EH2),-1,6.0,72,None,None,None,The arthropod tracheal system in the context o...,Wipfler,Hammel,30 keV,NaN,NaN
11016666,BAG-20211054,Microtomography (EH2),-1,6.0,72,None,None,None,High-sensitivity quantitative phase-contrast t...,Herzen,Hammel,30 keV,NaN,NaN
11016665,BAG-20211054,Microtomography (EH2),-1,12.0,72,None,None,None,High-sensitivity quantitative phase-contrast t...,Herzen,Hammel,30 keV,NaN,NaN
11016667,BAG-20211055,Microtomography (EH2),-1,12.0,72,None,None,None,The arthropod tracheal system in the context o...,Wipfler,Hammel,25 keV,NaN,NaN
11016664,BAG-20210019,Microtomography (EH2),-1,6.0,72,None,None,None,Understanding different scales oft the ecosyst...,Wipfler,Hammel,18 keV,NaN,NaN
11016060,I-20221037,Microtomography (EH2),1,9.0,9,None,None,None,In vivo imaging of the functional morphology o...,Hammel,Naumann,30-33,May to July,January to March (university teachings)
11016412,I-20221356,Nanotomography (EH1),2,12.0,12,None,None,None,Moisture dependent 3D shape respond of monocot...,Hesse,Hesse,17kev,13.01.2023,none


## Colorize schedule

In [54]:
def color_weekends(date):
    if date.weekday() == 5 :
        color = 'blue'
    elif date.weekday() == 6:
        color = 'mediumblue'
    else:
        color = None
    return 'background-color: %s' % color

def color_p3modes(mode):
    if mode in ['multi', 40]:
        color = 'lime'
    elif mode in ['tr']:
        color = 'green'
    else:
        color = 'red'
    return 'background-color: %s' % color

def colorize_schedule(styler):
    styler.set_caption('P05 schedule %s' % year)
    styler.applymap_index(color_weekends)
    styler.applymap(color_p3modes, subset='p3 modes')
    styler.format({"date": "{:%Y-%m-%d}"})  # Remove the time from the date which is introduced by styler
    styler.format_index("{:%Y-%m-%d}")  # Remove the time from the date which is introduced by styler
    return styler

In [55]:
p05_schedule_color = p5schclass.p05_schedule.style.pipe(colorize_schedule)
p05_schedule_color

,holidays,p3 modes,p3 info,type,Prop ID,app ID,leader,pi,local contact,absence,extra hours,short title,conferences,comment
2023-01-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-02,nan,A,CAV,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-03,nan,A,CAV,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-04,nan,A,CAV,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-05,nan,A,CAV PU25,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-06,nan,A,CAV PU25,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-07,nan,A,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-08,nan,A,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-09,nan,A,IEV PU25,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2023-01-10,nan,A,IEV PU25,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Test zone

In [42]:
with open("/home/fwilde/tmp/test.html", "w") as f:
    f.write(html)

FileNotFoundError: [Errno 2] No such file or directory: '/home/fwilde/tmp/test.html'

In [ ]:
with pd.ExcelWriter("/home/fwilde/tmp/test.ods") as writer:
    p05_schedule_color.to_excel(writer, sheet_name="p05_schedule", index=False)
    proposals.to_excel(writer, sheet_name="proposals_1", index=False)

In [ ]:
p05_schedule_color.to_excel("/home/fwilde/tmp/test.xlsx", sheet_name="p05_schedule", index=False)

In [134]:
init_school_holiday_column(school_holidays, 2023)

2023-01-01 00:00:00 HNS
2023-01-02 00:00:00 HNS
2023-01-03 00:00:00 HNS
2023-01-04 00:00:00 HNS
2023-01-05 00:00:00 HNS
2023-01-06 00:00:00 HNS
2023-01-07 00:00:00 S
2023-01-08 00:00:00 
2023-01-09 00:00:00 
2023-01-10 00:00:00 
2023-01-11 00:00:00 
2023-01-12 00:00:00 
2023-01-13 00:00:00 
2023-01-14 00:00:00 
2023-01-15 00:00:00 
2023-01-16 00:00:00 
2023-01-17 00:00:00 
2023-01-18 00:00:00 
2023-01-19 00:00:00 
2023-01-20 00:00:00 
2023-01-21 00:00:00 
2023-01-22 00:00:00 
2023-01-23 00:00:00 
2023-01-24 00:00:00 
2023-01-25 00:00:00 
2023-01-26 00:00:00 
2023-01-27 00:00:00 H
2023-01-28 00:00:00 
2023-01-29 00:00:00 
2023-01-30 00:00:00 N
2023-01-31 00:00:00 N
2023-02-01 00:00:00 
2023-02-02 00:00:00 
2023-02-03 00:00:00 
2023-02-04 00:00:00 
2023-02-05 00:00:00 
2023-02-06 00:00:00 
2023-02-07 00:00:00 
2023-02-08 00:00:00 
2023-02-09 00:00:00 
2023-02-10 00:00:00 
2023-02-11 00:00:00 
2023-02-12 00:00:00 
2023-02-13 00:00:00 
2023-02-14 00:00:00 
2023-02-15 00:00:00 
2023-02-16 0